## Connect to Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Connect to hugging face

Get your hugging face token from your Settings -> Access Tokens 😀

In [ ]:
!pip install -q huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 4.4 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

# Fine-tune Longformer Encoder-Decoder (LED) model on legal data

## Imports

In [ ]:
!pip install -q datasets
!pip install -q accelerate
!pip install -q transformers
!pip install -q nltk
!pip install -q evaluate
!pip install -q rouge_score
!pip install -q deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 836.6/836.6 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 17.9 MB/s eta 0:00:00


Run the cell below and restart runtime if you get error:  
```
Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`
```

In [ ]:
# !pip install -q --force-reinstall -v "accelerate==0.20.3"

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import json
import random
import nltk
nltk.download('punkt')

from IPython.display import display, HTML
import torch
import datasets
from datasets import load_dataset, load_metric, Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import LEDTokenizer, LEDForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
model_checkpoint = "allenai/led-large-16384"

## Split train and validation set

In [ ]:
import pandas as pd
extractive_output = pd.read_csv("/content/drive/MyDrive/W266 Final Project/output/train_data_1000.csv")
extractive_output = extractive_output[['Index', 'Summary', 'ExtractiveSummary']]

In [ ]:
# split data into training and validation dataset:
import pandas as pd
from sklearn.model_selection import train_test_split
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(extractive_output, test_size=0.2, random_state=42)
# Print the shapes of the resulting datasets
print("Training data shape:", train_df.shape)
print("Validation data shape:", val_df.shape)

Training data shape: (800, 3)
Validation data shape: (200, 3)


## Preprocess the training data

Create train, validate, and test set, each having a list of `document`, `summary`, and `id` in it.

In [ ]:
from transformers import LEDForConditionalGeneration, LEDTokenizer

# load tokenizer, using the same tokenizer from pretrained base
tokenizer = LEDTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_input_length = 16368
max_target_length = 1024
prefix = "summarize: "
def preprocess_function(example):
    inputs = [prefix + doc for doc in example["ExtractiveSummary"]]
    # tokenizer for input
    model_inputs = tokenizer(inputs, max_length=max_input_length, return_tensors="pt", truncation=True, padding=True)
    # tokenizer for reference summary
    labels = tokenizer(text_target=example["Summary"], max_length=max_target_length, truncation = True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# DEBUGGING ONLY, UPDATE TO THE REAL TRAIN/VAL/TEST SETS

# Splitting the dataframe
train_df = extractive_output.iloc[:10]
validation_df = extractive_output.iloc[10:12]
test_df = extractive_output.iloc[12:16]

In [ ]:
from datasets import Dataset

# Convert the DataFrames to the HuggingFace Dataset format
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# Combine them into a DatasetDict

from datasets import DatasetDict

dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})

In [ ]:
tokenized_datasets = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

## Fine-tune the model

Using **DeepSpeed** to configure ZeroOptimization to offload to CPU memory so that GPU memory utilization can be reduced. https://huggingface.co/blog/accelerate-deepspeed

https://huggingface.co/docs/transformers/main_classes/deepspeed


In [ ]:
# import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:35000'

# %env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:5000

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint).to("cuda").half()

In [ ]:
# // {
# //     "train_batch_size": 1, // The total batch size for training.
# //     "train_micro_batch_size_per_gpu": 1, // How many samples per GPU to load at a time. The actual batch size that each GPU sees is this value.
# //     "steps_per_print": 1, // How often to print training statistics/logs.
# //     "gradient_accumulation_steps": 1, // Number of steps to accumulate gradients before performing an optimization step (i.e., model parameter update).
# //     "optimizer": {
# //         "type": "Adam",
# //         "params": {
# //             "lr": 3e-4, // Learning rate.
# //             "betas": [0.9, 0.999], // Coefficients used for computing running averages of gradient and its square.
# //             "eps": 1e-8, // Term added to the denominator to improve numerical stability.
# //             "weight_decay": 3e-7 // Weight decay (L2 penalty).
# //         }
# //     },
# //     "fp16": {
# //         "enabled": true,
# //         "loss_scale": 0, // The initial scale factor to use. Setting to 0 lets the loss scaler determine an initial loss scale automatically.
# //         "loss_scale_window": 100, // How often to check and adjust the loss scale.
# //         "hysteresis": 2, // Number of consecutive times the loss scale can be halved without triggering a reduction in the number of times it's checked
# //         "min_loss_scale": 1 // The minimum loss scale that can be used.
# //     },
# //     // "zero_optimization": {
# //     //     "stage": 1,
# //     //     "offload_optimizer": {
# //     //         "device": "cpu",
# //     //         "pin_memory": true
# //     //     }
# //     // }
# //     "zero_optimization": {
# //         "stage": 2, // ZeRO optimization stage (1, 2, or 3). Each stage reduces memory requirements further.
# //         "allgather_partitions": true, //  If true, use allgather to collect updated weights among devices, otherwise use broadcast.
# //         "allgather_bucket_size": 5e8, // Determines the bucket size for allgather. A larger value might be faster as there are fewer kernel calls, but it also might be less memory efficient.
# //         "overlap_comm": true, // Overlap the reduction (communication) of gradients with the backward computation.
# //         "reduce_scatter": true, // Use reduce/scatter instead of reduce for communication.
# //         "reduce_bucket_size": 5e8, // Bucket size for reduction.
# //         "contiguous_gradients": true, // Make gradients contiguous in memory.
# //         "cpu_offload": true // Offload optimizer memory to CPU.
# //     }
# // }

In [ ]:
import accelerate
import transformers

batch_size = 1
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-legal",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    # deepspeed="ds_config.json"
)

TypeError: ignored

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np
from evaluate import load
metric = load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

NameError: ignored

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: ignored

In [ ]:
trainer.push_to_hub()